In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import Quine_McCluskey_Functions as qm

In [23]:
df = pd.read_csv("DFF_result.csv")

# Clean data.

# Pulse width values for Pass cases & Current percentage values for Fail cases. 
df.loc[df['fail'] == 'pass', 'Junction Values'] = df.loc[df['fail'] == 'pass', 'Pulse width info']
df.loc[df['fail'] == 'fail', 'Junction Values'] = df.loc[df['fail'] == 'fail', 'Current percentage']


# One-hot encode pass/fail.
drop_col = ['cus', 'real_flux', 'Faulty Junction', 'Pulse magnitude info', 'Phase state info', 'Pulse width info', 'Current percentage' , 'real_output', 'gold_output', 'pattern', 'MC']
df = df.drop(drop_col, axis=1)

# df.head()

In [24]:
# Rule 1: Initial Bias is between 60%-80%. 
# Rule 2: Junction 4 bias <55%. 

# Create rule 1, rule 2, pass/fail truth table.

for index, row in df.iterrows():
    val = eval(str(row['Junction Values']))
    if .6 <= val['B01RX1'] <.8:
        df.at[index, 'Rule 1'] = 1
    else: 
        df.at[index, 'Rule 1'] = 0
    if val['B4'] <.55:
        df.at[index, 'Rule 2'] = 1
    else: 
        df.at[index, 'Rule 2'] = 0

columns = df.columns.tolist()
df["fail"] = df["fail"].replace({"pass": 1, "fail": 0})
df = df[columns]

df.head(20)

,config,fail,Junction Values,Rule 1,Rule 2
0,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6...",0.0,0.0
1,and_spb_0_dff_spa_0_and,0,"{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7...",0.0,0.0
2,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.79, 'B01RX2': 0.69, 'B01TX1': 0.6...",1.0,1.0
3,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6...",1.0,1.0
4,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.77, 'B01RX2': 0.67, 'B01TX1': 0.6...",1.0,1.0
5,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6...",1.0,1.0
6,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.81, 'B01RX2': 0.7, 'B01TX1': 0.68...",0.0,0.0
7,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.78, 'B01RX2': 0.69, 'B01TX1': 0.6...",1.0,1.0
8,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6...",0.0,0.0
9,and_spb_0_dff_spa_0_and,1,"{'B01RX1': 0.82, 'B01RX2': 0.71, 'B01TX1': 0.6...",0.0,0.0


In [25]:
# Create buckets.
condition0 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 0]).all(axis=1)
condition0_df = df[condition0]
print(len(condition0_df))

condition1 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 1]).all(axis=1)
condition1_df = df[condition1]
print(len(condition1_df))

condition2 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 0]).all(axis=1)
condition2_df = df[condition2]
print(len(condition2_df))

condition3 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 1]).all(axis=1)
condition3_df = df[condition3]
print(len(condition3_df))

condition4 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 0]).all(axis=1)
condition4_df = df[condition4]
print(len(condition4_df))

condition5 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 1]).all(axis=1)
condition5_df = df[condition5]
print(len(condition5_df))

condition6 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 0]).all(axis=1)
condition6_df = df[condition6]
print(len(condition6_df))

condition7 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 1]).all(axis=1)
condition7_df = df[condition7]
print(len(condition7_df))

10105
5912
234
465
172
25
6202
16886


In [26]:
condition6_df['driver'] = condition6_df['config'].str.split('_').str[0]
condition6_df['load'] = condition6_df['config'].str.split('_').str[6]
condition6_df['splitterNum'] = condition6_df['config'].str.split('_').str[2]

condition6_df= pd.get_dummies(condition6_df, columns = ["driver", "load", "splitterNum"])

# condition6_df.head(20)

truthTable_6 = condition6_df[condition6_df.columns[-14:]]
# truthTable_6

minterms_6 = []
for row in truthTable_6.values:
    binary = "".join(map(str, row))
    minterms_6.append(binary)

minterms_6 = list(set(minterms_6))
minterms_6.sort()
print (len(minterms_6))
no_of_variable = 14
prime_implicants = qm.check(minterms_6)
print("Prime Implicants are:")
print(prime_implicants)
chart = qm.prime_implicant_chart(prime_implicants, minterms_6)

essential_prime_implicants = qm.selection(chart, prime_implicants)
print("Essential Prime Implicants are:")
print(essential_prime_implicants)
print(len(essential_prime_implicants))

truthTable_6.head()

for val in essential_prime_implicants:
    indices = qm.getIndices(val)
    
    values = []
    for val in indices:
        values.append (truthTable_6.columns[val])
    
    output = ' '.join(values)
    print (output, " + ")

100
Prime Implicants are:
['00001000010001', '00001000010010', '00001000010100', '00001000011000', '00001000100001', '00001000100010', '00001000100100', '00001000101000', '00001001000001', '00001001000010', '00001001000100', '00001001001000', '00001010000001', '00001010000010', '00001010000100', '00001010001000', '00001100000001', '00001100000010', '00001100000100', '00001100001000', '00010000010001', '00010000010010', '00010000010100', '00010000011000', '00010000100001', '00010000100010', '00010000100100', '00010000101000', '00010001000001', '00010001000010', '00010001000100', '00010001001000', '00010010000001', '00010010000010', '00010010000100', '00010010001000', '00010100000001', '00010100000010', '00010100000100', '00010100001000', '00100000010001', '00100000010010', '00100000010100', '00100000011000', '00100000100001', '00100000100010', '00100000100100', '00100000101000', '00100001000001', '00100001000010', '00100001000100', '00100001001000', '00100010000001', '00100010000010', '

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_32884\446865751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition6_df['driver'] = condition6_df['config'].str.split('_').str[0]
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_32884\446865751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition6_df['load'] = condition6_df['config'].str.split('_').str[6]
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_32884\446865751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

AttributeError: module 'Quine_McCluskey_Functions' has no attribute 'getIndices'

In [11]:
trial = ['00001000010001', '00001000010010', '00001000010100', '00001000011000', '00001000100001', '00001000100010', '00001000100100', '00001000101000', '00001001000001', '00001001000010', '00001001000100', '00001001001000', '00001010000001', '00001010000010', '00001010000100', '00001010001000', '00001100000001', '00001100000010', '00001100000100', '00001100001000', '00010000010001', '00010000010010', '00010000010100', '00010000011000', '00010000100001', '00010000100010', '00010000100100', '00010000101000', '00010001000001', '00010001000010', '00010001000100', '00010001001000', '00010010000001', '00010010000010', '00010010000100', '00010010001000', '00010100000001', '00010100000010', '00010100000100', '00010100001000', '00100000010001', '00100000010010', '00100000010100', '00100000011000', '00100000100001', '00100000100010', '00100000100100', '00100000101000', '00100001000001', '00100001000010', '00100001000100', '00100001001000', '00100010000001', '00100010000010', '00100010000100', '00100010001000', '00100100000001', '00100100000010', '00100100000100', '00100100001000', '01000000010001', '01000000010010', '01000000010100', '01000000011000', '01000000100001', '01000000100010', '01000000100100', '01000000101000', '01000001000001', '01000001000010', '01000001000100', '01000001001000', '01000010000001', '01000010000010', '01000010000100', '01000010001000', '01000100000001', '01000100000010', '01000100000100', '01000100001000', '10000000010001', '10000000010010', '10000000010100', '10000000011000', '10000000100001', '10000000100010', '10000000100100', '10000000101000', '10000001000001', '10000001000010', '10000001000100']
trial = [qm.convertOneHotBack(val, trial) for val in trial]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Carrie Lei\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_32884\800201898.py", line 2, in <cell line: 2>
    trial = [qm.convertOneHotBack(val) for val in trial]
  File "C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_32884\800201898.py", line 2, in <listcomp>
    trial = [qm.convertOneHotBack(val) for val in trial]
  File "c:\Users\Carrie Lei\Documents\CURVE\DelaySimulation\RSFQ_2\Quine_McCluskey_Functions.py", line 149, in convertOneHotBack
IndexError: string index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Carrie Lei\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB

In [ ]:
condition7_df['driver'] = condition7_df['config'].str.split('_').str[0]
condition7_df['load'] = condition7_df['config'].str.split('_').str[6]
condition7_df['splitterNum'] = condition7_df['config'].str.split('_').str[2]
condition7_df = pd.get_dummies(condition7_df, columns = ["driver", "load", "splitterNum"])

truthTable_7 = condition7_df.loc[:, 9:]

minterms_7 = []
for row in truthTable_7.values:
    binary = "".join(map(str, row))
    minterms_7.append(binary)

minterms_7 = list(set(minterms_7))
minterms_7.sort()
print(len(minterms_7))

no_of_variable = 14
prime_implicants = qm.check(minterms_7)
print("Prime Implicants are:")
print(prime_implicants)
chart = qm.prime_implicant_chart(prime_implicants, minterms_7)

essential_prime_implicants = qm.selection(chart, prime_implicants)
print("Essential Prime Implicants are:")
print(essential_prime_implicants)
print(len(essential_prime_implicants))

truthTable_7.head()
condition7_df.head()

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_27984\4263110134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition7_df['driver'] = condition7_df['config'].str.split('_').str[0]
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_27984\4263110134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition7_df['load'] = condition7_df['config'].str.split('_').str[6]
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_27984\4263110134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

75
Prime Implicants are:
['0000100001001', '0000100001010', '0000100001100', '0000100010001', '0000100010010', '0000100010100', '0000100100001', '0000100100010', '0000100100100', '0000101000001', '0000101000010', '0000101000100', '0000110000001', '0000110000010', '0000110000100', '0001000001001', '0001000001010', '0001000001100', '0001000010001', '0001000010010', '0001000010100', '0001000100001', '0001000100010', '0001000100100', '0001001000001', '0001001000010', '0001001000100', '0001010000001', '0001010000010', '0001010000100', '0010000001001', '0010000001010', '0010000001100', '0010000010001', '0010000010010', '0010000010100', '0010000100001', '0010000100010', '0010000100100', '0010001000001', '0010001000010', '0010001000100', '0010010000001', '0010010000010', '0010010000100', '0100000001001', '0100000001010', '0100000001100', '0100000010001', '0100000010010', '0100000010100', '0100000100001', '0100000100010', '0100000100100', '0100001000001', '0100001000010', '0100001000100', '0100

,config,pattern,MC,Rule 1,Rule 2,fail,real_output,gold_output,Junction Values,driver_and,...,driver_or,driver_xor,load_and,load_dff,load_inv,load_or,load_xor,splitterNum_0,splitterNum_1,splitterNum_2
2,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,0.01",1.0,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.79, 'B01RX2': 0.69, 'B01TX1': 0.6...",1,...,0,0,1,0,0,0,0,1,0,0
3,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.06,0.05",1.0,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6...",1,...,0,0,1,0,0,0,0,1,0,0
4,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.03",1.0,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.77, 'B01RX2': 0.67, 'B01TX1': 0.6...",1,...,0,0,1,0,0,0,0,1,0,0
5,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.05",1.0,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6...",1,...,0,0,1,0,0,0,0,1,0,0
7,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.01,0.01",1.0,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.78, 'B01RX2': 0.69, 'B01TX1': 0.6...",1,...,0,0,1,0,0,0,0,1,0,0
